In [ ]:
import warnings
import datetime as dt
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time
    Src['Week Day']                     = Query['datetime'].dt.day_name().map(WEEK_DAY)

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [17]:
Calc.groupby('Week Day')[['Change']].describe().round(0).astype(int)

Change                                     
          count mean   std   min  25% 50%  75%   max
Week Day                                            
2           247   12  1303 -5150 -772  80  875  4440
3           250   22  1247 -3380 -764  58  889  4170
4           252  -54  1280 -3580 -881  15  732  4535
5           249   -2  1423 -4000 -975  70  895  3650
6           247  -85  1312 -3600 -970 -90  700  3985

In [18]:
Calc.groupby('Week Day')[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                     
           count  mean  std min  25%  50%   75%   max
Week Day                                             
2            247   992  843   5  420  840  1325  5150
3            250   980  769   0  420  838  1319  4170
4            252   998  801   5  395  795  1400  4535
5            249  1128  865   0  435  955  1650  4000
6            247  1055  782   5  435  825  1595  3985

In [19]:
Calc.groupby('Week Day')[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                     
           count  mean  std min  25%  50%   75%   max
Week Day                                             
2            133   931  772   5  395  800  1265  4440
3            131   956  764   0  318  860  1370  4170
4            127   937  774  10  380  725  1340  4535
5            133  1054  825   0  400  860  1580  3650
6            118  1015  796   5  398  762  1524  3985

In [20]:
Calc.groupby('Week Day')[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                      
           count  mean  std   min   25%   50%  75% max
Week Day                                              
2            114 -1062  917 -5150 -1365  -852 -496 -15
3            119 -1007  776 -3380 -1242  -800 -478  -5
4            125 -1060  825 -3580 -1435  -885 -430  -5
5            116 -1213  905 -4000 -1701 -1018 -491 -10
6            129 -1091  770 -3600 -1690  -960 -480 -10

In [21]:
Calc.groupby('Week Day')[['Hilo']].describe().round(0).astype(int)

Hilo                                        
         count  mean  std  min   25%   50%   75%   max
Week Day                                              
2          247  1869  815  535  1295  1700  2218  5950
3          250  1927  753  610  1388  1705  2379  5300
4          252  2005  835  630  1474  1822  2326  7400
5          249  2060  856  660  1425  1895  2505  4975
6          247  2028  780  775  1465  1820  2422  5205

In [22]:
Calc.groupby('Week Day')[['HL Pos']].describe().round(0).astype(int)

HL Pos                                        
          count  mean  std  min   25%   50%   75%   max
Week Day                                               
2           133  1802  795  535  1250  1670  2135  5950
3           131  1843  755  835  1352  1590  2195  5300
4           127  1946  816  630  1468  1805  2295  7400
5           133  1981  811  660  1365  1845  2505  4430
6           118  2002  753  930  1475  1828  2382  4545

In [23]:
Calc.groupby('Week Day')[['HL Neg']].describe().round(0).astype(int)

HL Neg                                        
          count  mean  std   min   25%   50%   75%  max
Week Day                                               
2           114 -1946  836 -5720 -2325 -1738 -1356 -665
3           119 -2019  744 -4335 -2502 -1805 -1450 -610
4           125 -2064  852 -5225 -2440 -1895 -1500 -830
5           116 -2150  900 -4975 -2515 -1980 -1596 -675
6           129 -2052  807 -5205 -2425 -1775 -1455 -775

# Snippets

In [24]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }
    Src = pd.DataFrame()
    Src['Week Day']     = Query['datetime'].dt.day_name().map(WEEK_DAY)
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc